# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

Using Yfinance unlike most set up tutorials that use IEX Sandbox, which has been since deprecated

In [103]:
import numpy as np
import pandas as pd
import requests
import yfinance as yf
from scipy import stats
from datetime import timedelta, datetime
import math

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [104]:
df = pd.read_csv('constituents.csv')
Tickers = df['Symbol']
Tickers

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

## Defining one year return for list of tickers in this case S&P 500 stocks

We need to get one-year price returns for each stock in the S&P 500, trying to finalize the same.  

In [105]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def calculate_one_year_returns(symbols):
    """
    Calculate one-year price returns for a list of stock symbols using Yahoo Finance.
    
    Parameters:
    symbols (list): List of stock symbols to fetch data for.
    
    Returns:
    pd.DataFrame: DataFrame containing the one-year price returns for the given symbols.
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    returns_data = {}

    for symbol in symbols:
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date, end=end_date)
        except:
            continue
        if not data.empty:
            start_price = data['Close'].iloc[0]
            end_price = data['Close'].iloc[-1]
            one_year_return = (end_price - start_price) / start_price
            returns_data[symbol] = one_year_return
        else:
            returns_data[symbol] = None

    returns_df = pd.DataFrame(list(returns_data.items()), columns=['Symbol', 'One_Year_Return'])
    returns_df['One_Year_Return'] = returns_df['One_Year_Return'].apply(lambda x: x * 100 if x is not None else x)  # Convert to percentage

    return returns_df
# Example usage

In [106]:
returns_df = calculate_one_year_returns(Tickers)
returns_df

$BRK.B: possibly delisted; No timezone found
$BF.B: possibly delisted; No price data found  (1d 2023-07-30 14:45:26.564686 -> 2024-07-29 14:45:26.564686)


$BF.B: possibly delisted; No price data found  (1d 2023-07-30 14:45:26.564686 -> 2024-07-29 14:45:26.564686)


,Symbol,One_Year_Return
0,MMM,36.397636
1,AOS,16.343113
2,ABT,-3.504833
3,ABBV,28.569276
4,ACN,5.505222
...,...,...
498,XYL,26.424591
499,YUM,-5.176355
500,ZBRA,5.851416
501,ZBH,-18.774411


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [107]:
# Function to split the list into chunks of a specified size
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Example my_columns for output structure
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [108]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

def get_one_year_price_return(tickers: list) -> dict:
    # Define the date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    # Fetch the historical data for multiple tickers
    data = yf.download(tickers, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    
    # Initialize a dictionary to store returns
    returns = {}
    
    # If fetching multiple tickers, data will have a multi-level column index
    if isinstance(data.columns, pd.MultiIndex):
        for ticker in tickers:
            try:
                closing_prices = data['Close'][ticker]
                if len(closing_prices) > 0:
                    start_price = closing_prices.iloc[0]
                    end_price = closing_prices.iloc[-1]
                    one_year_return = (end_price - start_price) / start_price * 100
                    returns[ticker] = one_year_return
                else:
                    returns[ticker] = None  # Handle case where there might be no data
            except KeyError:
                returns[ticker] = None  # Handle case where ticker data is not available
    else:
        # Handle the case where data is for a single ticker
        closing_prices = data['Close']
        if len(closing_prices) > 0:
            start_price = closing_prices.iloc[0]
            end_price = closing_prices.iloc[-1]
            one_year_return = (end_price - start_price) / start_price * 100
            for ticker in tickers:
                returns[ticker] = one_year_return
        else:
            for ticker in tickers:
                returns[ticker] = None  # Handle case where there might be no data
    
    return returns

# Example usage
tickers = ["AAPL", "MSFT", "GOOGL", "TSLA", "AMZN", "META", "NFLX", "NVDA", "BABA", "V", "JPM", "DIS"]  # Add more tickers as needed
Tickers = Tickers.tolist()
returns = get_one_year_price_return(Tickers)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
print

[********************  42%%                      ]  213 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2023-07-30 -> 2024-07-29)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-07-30 -> 2024-07-29)')


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [109]:
final_df = pd.DataFrame(columns=my_columns)
rows=[]
for key, value in returns.items():
    row_df = pd.DataFrame([[key, None, value, None]], columns=my_columns)
    rows.append(row_df)
final_df = pd.concat(rows, ignore_index=True)

In [110]:
print(final_df)

    Ticker Price  One-Year Price Return Number of Shares to Buy
0      MMM  None              36.397636                    None
1      AOS  None              16.343113                    None
2      ABT  None              -5.470227                    None
3     ABBV  None              23.786603                    None
4      ACN  None               3.828034                    None
..     ...   ...                    ...                     ...
498    XYL  None              24.913522                    None
499    YUM  None              -6.987721                    None
500   ZBRA  None               5.851416                    None
501    ZBH  None             -19.442631                    None
502    ZTS  None              -4.386198                    None

[503 rows x 4 columns]


In [111]:
prices = []
for ticker in final_df['Ticker']:
    price = (yf.Ticker(ticker).info.get('previousClose'))
    prices.append(price)

In [112]:
final_df['Price'] = prices
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,127.16,36.397636,None
1,AOS,84.50,16.343113,None
2,ABT,105.24,-5.470227,None
3,ABBV,185.16,23.786603,None
4,ACN,328.46,3.828034,None
...,...,...,...,...
498,XYL,140.84,24.913522,None
499,YUM,128.05,-6.987721,None
500,ZBRA,325.98,5.851416,None
501,ZBH,111.29,-19.442631,None


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [113]:
final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,477,VST,71.12,153.456893,None
1,350,NVDA,113.06,141.948249,None
2,430,SMCI,712.19,115.638728,None
3,44,ANET,319.21,105.822425,None
4,279,KKR,118.51,99.578983,None
5,348,NRG,73.71,94.024732,None
6,191,FICO,1605.94,91.646479,None
7,222,GDDY,143.65,86.340642,None
8,210,GE,169.81,86.249930,None
9,125,CEG,175.04,81.107078,None


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [114]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input("Enter your portfolio Size") 
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Not a number')
        print('enter a number')
        portfolio_size = input("Enter your portfolio Size") 
portfolio_input()
print(portfolio_size)
    

100000


In [115]:
position_size = float(portfolio_size)/50
num_shares = []
for price in final_df['Price']:
    num_shares.append(math.floor(position_size/price))

In [116]:
final_df['Number of Shares to Buy'] = num_shares
final_df

/var/folders/g4/d2_9vd6n0cx4k51m5kmh8f2m0000gn/T/ipykernel_51946/2858503378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Number of Shares to Buy'] = num_shares


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,477,VST,71.12,153.456893,28
1,350,NVDA,113.06,141.948249,17
2,430,SMCI,712.19,115.638728,2
3,44,ANET,319.21,105.822425,6
4,279,KKR,118.51,99.578983,16
5,348,NRG,73.71,94.024732,27
6,191,FICO,1605.94,91.646479,1
7,222,GDDY,143.65,86.340642,13
8,210,GE,169.81,86.249930,11
9,125,CEG,175.04,81.107078,11


## Optimizing to match real world momentum strategies 
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

`hqm` = `high-quality momentum`.

In [117]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]
hqm_df = pd.DataFrame(columns = hqm_columns)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [118]:
def get_price_return(tickers: list, days: int) -> dict:
    # Define the date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    # Fetch the historical data for multiple tickers
    data = yf.download(tickers, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    
    # Initialize a dictionary to store returns
    returns = {}
    
    # If fetching multiple tickers, data will have a multi-level column index
    if isinstance(data.columns, pd.MultiIndex):
        for ticker in tickers:
            try:
                closing_prices = data['Close'][ticker]
                if len(closing_prices) > 0:
                    start_price = closing_prices.iloc[0]
                    end_price = closing_prices.iloc[-1]
                    one_year_return = (end_price - start_price) / start_price * 100
                    returns[ticker] = one_year_return
                else:
                    returns[ticker] = None  # Handle case where there might be no data
            except KeyError:
                returns[ticker] = None  # Handle case where ticker data is not available
    else:
        # Handle the case where data is for a single ticker
        closing_prices = data['Close']
        if len(closing_prices) > 0:
            start_price = closing_prices.iloc[0]
            end_price = closing_prices.iloc[-1]
            one_year_return = (end_price - start_price) / start_price * 100
            for ticker in tickers:
                returns[ticker] = one_year_return
        else:
            for ticker in tickers:
                returns[ticker] = None  # Handle case where there might be no data
    
    return returns

In [119]:
six_month = get_price_return(Tickers,180)
three_month = get_price_return(Tickers,90)
one_month = get_price_return(Tickers,30)


[********************  42%%                      ]  211 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2024-01-31 -> 2024-07-29)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-01-31 -> 2024-07-29)')
[********************  42%%                      ]  209 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2024-04-30 -> 2024-07-29)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-04-30 -> 2024-07-29)')
[********************  42%%                      ]  209 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2024-06-29 -> 2024-07-29)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-06-29 -> 2024-07-29)')


In [120]:
six_month

{'MMM': 61.19063021548447,
 'AOS': 8.877721086796969,
 'ABT': -6.99072342504587,
 'ABBV': 12.62774363525627,
 'ACN': -9.733981794119572,
 'ADBE': -12.19528364185175,
 'AMD': -16.51857388333037,
 'AES': 5.035972045679398,
 'AFL': 12.425899705932691,
 'A': 5.549577926198132,
 'APD': 2.0609300121594125,
 'ABNB': -2.802825935368326,
 'AKAM': -21.309747156436266,
 'ALB': -19.12148790351296,
 'ARE': -1.637719877538438,
 'ALGN': -17.533296832185975,
 'ALLE': 10.396323307988233,
 'LNT': 15.104805789496915,
 'ALL': 8.966182395455919,
 'GOOGL': 19.200565827676545,
 'GOOG': 18.956268720397762,
 'MO': 25.697910388270966,
 'AMZN': 17.5902084977889,
 'AMCR': 10.07422882843086,
 'AEE': 12.174789791372193,
 'AAL': -25.368937264982605,
 'AEP': 23.432298056837915,
 'AXP': 22.491776756488083,
 'AIG': 11.120695568947454,
 'AMT': 9.235884430325786,
 'AWK': 14.263836921164582,
 'AMP': 13.626146835131467,
 'AME': 6.306696098881847,
 'AMGN': 6.551898331972509,
 'APH': 26.330369088379406,
 'ADI': 17.7115782685

In [121]:
hqm_df['Ticker'] = final_df['Ticker']
hqm_df['Price'] = final_df['Price']
hqm_df['One-Year Price Return'] = final_df['One-Year Price Return']
hqm_df

for key, value in six_month.items():
    hqm_df.loc[hqm_df['Ticker'] == key, 'Six-Month Price Return'] = value
for key, value in one_month.items():
    hqm_df.loc[hqm_df['Ticker'] == key, 'One-Month Price Return'] = value
    
for key, value in three_month.items():
    hqm_df.loc[hqm_df['Ticker'] == key, 'Three-Month Price Return'] = value
    
hqm_df


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,VST,71.12,NaN,153.456893,NaN,73.336595,NaN,-6.223621,NaN,-18.571095,NaN
1,NVDA,113.06,NaN,141.948249,NaN,83.756719,NaN,30.853449,NaN,-9.042643,NaN
2,SMCI,712.19,NaN,115.638728,NaN,34.474429,NaN,-17.071494,NaN,-12.326424,NaN
3,ANET,319.21,NaN,105.822425,NaN,23.399567,NaN,24.419237,NaN,-10.510238,NaN
4,KKR,118.51,NaN,99.578983,NaN,36.879186,NaN,27.334267,NaN,13.395847,NaN
5,NRG,73.71,NaN,94.024732,NaN,38.970584,NaN,1.431128,NaN,-5.317918,NaN
6,FICO,1605.94,NaN,91.646479,NaN,33.958943,NaN,41.701005,NaN,7.312342,NaN
7,GDDY,143.65,NaN,86.340642,NaN,34.680282,NaN,17.380289,NaN,1.51943,NaN
8,GE,169.81,NaN,86.249930,NaN,60.679596,NaN,4.937579,NaN,6.818894,NaN
9,CEG,175.04,NaN,81.107078,NaN,43.475404,NaN,-5.862111,NaN,-14.635459,NaN


In [122]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for row in hqm_df.index:
    for period in time_periods:
        hqm_df.loc[row,f'{period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{period} Price Return'],hqm_df.loc[row,f'{period} Price Return'])
        

In [123]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,VST,71.12,NaN,153.456893,100.0,73.336595,98.0,-6.223621,8.0,-18.571095,4.0
1,NVDA,113.06,NaN,141.948249,98.0,83.756719,100.0,30.853449,96.0,-9.042643,18.0
2,SMCI,712.19,NaN,115.638728,96.0,34.474429,66.0,-17.071494,2.0,-12.326424,10.0
3,ANET,319.21,NaN,105.822425,94.0,23.399567,36.0,24.419237,86.0,-10.510238,16.0
4,KKR,118.51,NaN,99.578983,92.0,36.879186,76.0,27.334267,92.0,13.395847,88.0
5,NRG,73.71,NaN,94.024732,90.0,38.970584,78.0,1.431128,20.0,-5.317918,30.0
6,FICO,1605.94,NaN,91.646479,88.0,33.958943,64.0,41.701005,100.0,7.312342,70.0
7,GDDY,143.65,NaN,86.340642,86.0,34.680282,70.0,17.380289,70.0,1.51943,48.0
8,GE,169.81,NaN,86.249930,84.0,60.679596,96.0,4.937579,30.0,6.818894,66.0
9,CEG,175.04,NaN,81.107078,82.0,43.475404,82.0,-5.862111,10.0,-14.635459,8.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [124]:
from statistics import mean

hqm_df['HQM Score'] = 0

for row in hqm_df.index:
    momentums = []
    momentums.append(hqm_df.loc[row,'One-Month Return Percentile'])
    momentums.append(hqm_df.loc[row,'Six-Month Return Percentile'])
    momentums.append(hqm_df.loc[row,'Three-Month Return Percentile'])
    momentums.append(hqm_df.loc[row,'One-Year Return Percentile'])
    
    hqm_df.loc[row,'HQM Score'] = mean(momentums)

/var/folders/g4/d2_9vd6n0cx4k51m5kmh8f2m0000gn/T/ipykernel_51946/4240931402.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '52.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  hqm_df.loc[row,'HQM Score'] = mean(momentums)


In [125]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VST,71.12,NaN,153.456893,100.0,73.336595,98.0,-6.223621,8.0,-18.571095,4.0,52.5
1,NVDA,113.06,NaN,141.948249,98.0,83.756719,100.0,30.853449,96.0,-9.042643,18.0,78.0
2,SMCI,712.19,NaN,115.638728,96.0,34.474429,66.0,-17.071494,2.0,-12.326424,10.0,43.5
3,ANET,319.21,NaN,105.822425,94.0,23.399567,36.0,24.419237,86.0,-10.510238,16.0,58.0
4,KKR,118.51,NaN,99.578983,92.0,36.879186,76.0,27.334267,92.0,13.395847,88.0,87.0
5,NRG,73.71,NaN,94.024732,90.0,38.970584,78.0,1.431128,20.0,-5.317918,30.0,54.5
6,FICO,1605.94,NaN,91.646479,88.0,33.958943,64.0,41.701005,100.0,7.312342,70.0,80.5
7,GDDY,143.65,NaN,86.340642,86.0,34.680282,70.0,17.380289,70.0,1.51943,48.0,68.5
8,GE,169.81,NaN,86.249930,84.0,60.679596,96.0,4.937579,30.0,6.818894,66.0,69.0
9,CEG,175.04,NaN,81.107078,82.0,43.475404,82.0,-5.862111,10.0,-14.635459,8.0,45.5


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [126]:
hqm_df.sort_values('HQM Score',ascending=False,inplace=True)

In [127]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
4,KKR,118.51,NaN,99.578983,92.0,36.879186,76.0,27.334267,92.0,13.395847,88.0,87.0
13,GRMN,177.94,NaN,68.042311,74.0,48.916232,90.0,23.16744,80.0,10.453132,86.0,82.5
33,MHK,160.71,NaN,51.128467,34.0,54.15828,92.0,39.360048,98.0,46.232943,100.0,81.0
6,FICO,1605.94,NaN,91.646479,88.0,33.958943,64.0,41.701005,100.0,7.312342,70.0,80.5
22,IRM,98.50,NaN,60.423449,56.0,45.882709,86.0,27.063989,90.0,9.82272,82.0,78.5
1,NVDA,113.06,NaN,141.948249,98.0,83.756719,100.0,30.853449,96.0,-9.042643,18.0,78.0
20,HWM,82.68,NaN,61.673839,60.0,46.960545,88.0,23.865169,84.0,6.959893,68.0,75.0
36,TYL,590.65,NaN,48.917131,28.0,39.71615,80.0,27.970976,94.0,18.243521,96.0,74.5
27,UHS,213.69,NaN,53.778060,46.0,34.557021,68.0,25.382862,88.0,17.283209,92.0,73.5
8,GE,169.81,NaN,86.249930,84.0,60.679596,96.0,4.937579,30.0,6.818894,66.0,69.0


In [128]:
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True)
hqm_df

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,4,KKR,118.51,NaN,99.578983,92.0,36.879186,76.0,27.334267,92.0,13.395847,88.0,87.0
1,13,GRMN,177.94,NaN,68.042311,74.0,48.916232,90.0,23.16744,80.0,10.453132,86.0,82.5
2,33,MHK,160.71,NaN,51.128467,34.0,54.15828,92.0,39.360048,98.0,46.232943,100.0,81.0
3,6,FICO,1605.94,NaN,91.646479,88.0,33.958943,64.0,41.701005,100.0,7.312342,70.0,80.5
4,22,IRM,98.50,NaN,60.423449,56.0,45.882709,86.0,27.063989,90.0,9.82272,82.0,78.5
5,1,NVDA,113.06,NaN,141.948249,98.0,83.756719,100.0,30.853449,96.0,-9.042643,18.0,78.0
6,20,HWM,82.68,NaN,61.673839,60.0,46.960545,88.0,23.865169,84.0,6.959893,68.0,75.0
7,36,TYL,590.65,NaN,48.917131,28.0,39.71615,80.0,27.970976,94.0,18.243521,96.0,74.5
8,27,UHS,213.69,NaN,53.778060,46.0,34.557021,68.0,25.382862,88.0,17.283209,92.0,73.5
9,8,GE,169.81,NaN,86.249930,84.0,60.679596,96.0,4.937579,30.0,6.818894,66.0,69.0


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [129]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input("Enter your portfolio Size") 
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Not a number')
        print('enter a number')
        portfolio_size = input("Enter your portfolio Size") 
portfolio_input()
print(portfolio_size)

position_size = float(portfolio_size)/50
num_shares = []
for price in final_df['Price']:
    num_shares.append(math.floor(position_size/price))
hqm_df['Number of Shares to Buy'] = num_shares    

1000000


/var/folders/g4/d2_9vd6n0cx4k51m5kmh8f2m0000gn/T/ipykernel_51946/18854125.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_df['Number of Shares to Buy'] = num_shares


In [130]:
print(hqm_df)

    index Ticker    Price  Number of Shares to Buy  One-Year Price Return  \
0       4    KKR   118.51                      281              99.578983   
1      13   GRMN   177.94                      176              68.042311   
2      33    MHK   160.71                       28              51.128467   
3       6   FICO  1605.94                       62              91.646479   
4      22    IRM    98.50                      168              60.423449   
5       1   NVDA   113.06                      271             141.948249   
6      20    HWM    82.68                       12              61.673839   
7      36    TYL   590.65                      139              48.917131   
8      27    UHS   213.69                      117              53.778060   
9       8     GE   169.81                      114              86.249930   
10      7   GDDY   143.65                       24              86.340642   
11     19   NTAP   126.15                       92              61.710035   

## Formatting Our Excel Output


In [133]:
hqm_df
hqm_df.to_excel('QMTStrategy.xlsx', engine='openpyxl', index=False)

## Saving Our Excel Output

As before, saving our Excel output is very easy: